# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [36]:
# import libraries

#General numeric and dataframes
import numpy as np
import pandas as pd

# Data base related
from sqlalchemy import create_engine

#Regex
import re

#Object serialization
import pickle

#Natural Language
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

#Machine learning models
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier

from sklearn.metrics import classification_report, f1_score, accuracy_score, precision_score, log_loss, recall_score


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [37]:


# load data from database https://www.accenture.com/_acnmedia/PDF-92/Accenture-Insurance-Digital-Transformation-Remaking-an-Industry.pdf#zoom=50
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('MessagesCategories', engine)
X = df["message"]
Y = df.loc[:, "related":"direct_report"]

### 2. Write a tokenization function to process your text data

In [38]:
def tokenize(text):
    
    stop_words = stopwords.words('english')
    lemmatizer = WordNetLemmatizer()
    
    #remove punctation
    text = re.sub(r"[^a-zA-Z0-9]",' ',text.lower())
    
    #tokenize text
    tokens = word_tokenize(text)
    
    #lemmatize and remove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]    
    
    return (tokens)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [39]:
steps = [
    ('tfidf',TfidfVectorizer(tokenizer=tokenize)),
    ('clf',MultiOutputClassifier(AdaBoostClassifier(random_state=1) ))
]
 
pipeline = Pipeline(steps)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [40]:
X_train, X_test,y_train, y_test = train_test_split(X, Y, test_size=0.3,random_state=1)


In [41]:
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...stimator=None,
          learning_rate=1.0, n_estimators=50, random_state=1),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [42]:

y_pred = pipeline.predict(X_test)
category_names = y_test.columns.tolist()
y_pred = pd.DataFrame(y_pred, columns = category_names)




    
print("\n Classification Report")
for i in range(36):
    print(category_names[i] , classification_report(y_test.iloc[:,i], y_pred.iloc[:,i]))



 Classification Report
related              precision    recall  f1-score   support

          0       0.68      0.09      0.16      1868
          1       0.77      0.99      0.86      5937
          2       0.18      0.03      0.06        60

avg / total       0.74      0.77      0.69      7865

request              precision    recall  f1-score   support

          0       0.90      0.96      0.93      6511
          1       0.74      0.51      0.60      1354

avg / total       0.88      0.88      0.88      7865

offer              precision    recall  f1-score   support

          0       1.00      1.00      1.00      7832
          1       0.11      0.03      0.05        33

avg / total       0.99      0.99      0.99      7865

aid_related              precision    recall  f1-score   support

          0       0.75      0.86      0.80      4577
          1       0.76      0.59      0.66      3288

avg / total       0.75      0.75      0.74      7865

medical_help              pre

### 6. Improve your model
Use grid search to find better parameters. 

In [43]:
pipeline.get_params()

{'memory': None,
 'steps': [('tfidf',
   TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
           stop_words=None, strip_accents=None, sublinear_tf=False,
           token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f5e71028d08>, use_idf=True,
           vocabulary=None)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=1),
              n_jobs=1))],
 'tfidf': TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1, 1), nor

In [44]:
parameters = {
    'tfidf__smooth_idf': [True, False]
}

cv = GridSearchCV(pipeline,param_grid=parameters,cv=5,n_jobs=-1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [45]:
cv.fit(X_train, y_train)
cv.best_score_
y_pred = cv.predict(X_test)

y_pred = pd.DataFrame(y_pred, columns = category_names)
y_pred.head()


,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
1,1,1,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,1
2,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
print("\n Classification Report")
for i in range(36):
    print("\n" + category_names[i] +"\n", classification_report(y_test.iloc[:,i], y_pred.iloc[:,i]))
    
cv.best_params_  


 Classification Report

related
              precision    recall  f1-score   support

          0       0.68      0.09      0.16      1868
          1       0.77      0.99      0.86      5937
          2       0.18      0.03      0.06        60

avg / total       0.74      0.77      0.69      7865


request
              precision    recall  f1-score   support

          0       0.90      0.96      0.93      6511
          1       0.74      0.51      0.60      1354

avg / total       0.87      0.88      0.87      7865


offer
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      7832
          1       0.15      0.06      0.09        33

avg / total       0.99      0.99      0.99      7865


aid_related
              precision    recall  f1-score   support

          0       0.75      0.86      0.80      4577
          1       0.75      0.60      0.67      3288

avg / total       0.75      0.75      0.75      7865


medical_help
       

{'tfidf__smooth_idf': False}

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [57]:

parameters = [{'tfidf__smooth_idf': [False],
                     'tfidf__max_df': [1.0, 2.0],
                     "clf": [MultiOutputClassifier(AdaBoostClassifier(random_state=1))],
                    #'clf__estimator__learning_rate': [0.5, 1],
                    'clf__estimator__n_estimators': [50, 100]
              },
              {'tfidf__smooth_idf': [False],
                    'tfidf__max_df': [1.0, 2.0],
                     "clf": [MultiOutputClassifier(RandomForestClassifier(random_state = 1))], 
                     'clf__estimator__max_features': ["auto", "log2"]
              }]


cv = GridSearchCV(pipeline, param_grid=parameters, cv = 5)


#Source https://chrisalbon.com/machine_learning/model_selection/model_selection_using_grid_search/
#http://www.davidsbatista.net/blog/2018/02/23/model_optimization/


In [58]:
cv.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...stimator=None,
          learning_rate=1.0, n_estimators=50, random_state=1),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'tfidf__smooth_idf': [False], 'tfidf__max_df': [1.0, 2.0], 'clf': [MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1,
          n_estimators=100, random_state=1),
           n_jobs=1)], 'clf__estimator__learning_rate': [0.5, 1],...bose=0, warm_start=False),
           n_jobs=1)], 'clf__estimator__max_features': ['auto', 'log2']}],
       pre_dispatch='2*n_jobs', refit=True, return_tra

In [60]:
                   
y_pred = cv.predict(X_test)
y_pred = pd.DataFrame(y_pred, columns = category_names)
y_pred.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
1,1,1,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [65]:
print("\n Classification Report")
for i in range(36):
    print(category_names[i] , classification_report(y_test.iloc[:,i], y_pred.iloc[:,i]))

cv.best_params_    


 Classification Report
related              precision    recall  f1-score   support

          0       0.64      0.48      0.55      1868
          1       0.84      0.91      0.88      5937
          2       1.00      0.07      0.12        60

avg / total       0.79      0.80      0.79      7865

request              precision    recall  f1-score   support

          0       0.89      0.97      0.93      6511
          1       0.77      0.43      0.55      1354

avg / total       0.87      0.88      0.87      7865

offer              precision    recall  f1-score   support

          0       1.00      1.00      1.00      7832
          1       0.00      0.00      0.00        33

avg / total       0.99      1.00      0.99      7865

aid_related              precision    recall  f1-score   support

          0       0.75      0.85      0.80      4577
          1       0.74      0.62      0.67      3288

avg / total       0.75      0.75      0.75      7865

medical_help              pre

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


{'clf': MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
             oob_score=False, random_state=1, verbose=0, warm_start=False),
            n_jobs=1),
 'clf__estimator__max_features': 'auto',
 'tfidf__max_df': 1.0,
 'tfidf__smooth_idf': False}

### 9. Export your model as a pickle file

In [62]:

filename = 'classifier.pkl'
pickle.dump(cv, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.